In [ ]:
#| default_exp core

In [ ]:
#| export
import ovh
from fastcore.basics import patch
from loguru import logger
from typing import Any

# Simple, direct imports - let Python handle the module resolution
try:
    from ovhmanager.project import OVHProject
    from ovhmanager.catalog import GPUCatalog
    from ovhmanager.instance import GPUInstance
except ImportError:
    # Fallback for development/notebook environment
    from .project import OVHProject
    from .catalog import GPUCatalog
    from .instance import GPUInstance

In [ ]:
#| export

class OVHGPUManager:
    def __init__(self, endpoint=None, application_key=None, application_secret=None, consumer_key=None):
        """
        OVHGPUManager is a OVH API client wrapper that enables to create, delete, start and stop GPU instances
        
        By default the OVH client expects the following environment variables to be set, but you can overwrite them using the provided arguments:
        
        - OVH_ENDPOINT
        - OVH_APPLICATION_KEY
        - OVH_APPLICATION_SECRET
        - OVH_CONSUMER_KEY

        Parameters
        ----------
        endpoint : str, optional
            API endpoint like 'ovh-eu'
        application_key : str, optional
            Application key
        application_secret : str, optional
            Application secret
        consumer_key : str, optional
            Consumer key
        """
        self.client = ovh.Client(
        endpoint=endpoint,
        application_key=application_key,
        application_secret=application_secret,
        consumer_key=consumer_key)

        self._base_catalog_url = "/order/catalog/public/cloud"
        self.region = None
        self.project = None
        self.consumption_mode = None
        self.catalog_dict = None
        self.catalog = None
        self.gpu_type = None
        self.gpu_flavor_id = None
        self.instance_image_id = None
        self.instance_image_name = None
        self.ssh_key_id = None
        self.ssh_key_name = None
        self.last_gpu_instance = None
        self.last_instance_dict = None
        self.gpu_instances = []

This is an example of OVHGPUManager creation, it assumes the expected environment variables have been set


In [ ]:
#| eval: false
# assuming environment variables are set
client = OVHGPUManager()

In [ ]:
#| export
@patch
def list_projects(self:OVHGPUManager)->list:
    """
    Returns the list of available project IDs for the OVH Cloud account
    """
    return self.client.get('/cloud/project')

In [ ]:
#| eval: false
# listing the projects existing in the OVH Cloud account
client.list_projects()

['PROJECT_ID']

In [ ]:
#| export
@patch
def select_project(self:OVHGPUManager, project_idx:str=None)->Any:
    """
    Lists available projects for the OVH Cloud account and prompts user to select a project if no project id is passed as an argument
    
    Parameters
    ----------
    project_idx : str, optional
        Project OVH ID
    """
    logger.info("Available projects:")
    projects = self.list_projects()
    for i, p in enumerate(projects):
        print(f"{i+1}. {p}")
    if not project_idx:
        idx = int(input("Select project number: ")) - 1
        project_idx = projects[idx]
    self.project = OVHProject(project_idx)
    logger.info(f"Selected project: {self.project.id}")
    return self.project

Let's now select a project by passing a specific project ID

In [ ]:
#| eval: false
client.select_project('PROJECT_ID')

2026-02-11 15:20:53.542 | INFO     | __main__:select_project:12 - Available projects:
2026-02-11 15:20:53.586 | INFO     | __main__:select_project:20 - Selected project: PROJECT_ID


1. PROJECT_ID


<ovhmanager.project.OVHProject>

Or by interactively selecting it

In [ ]:
#| eval: false
client.select_project()

2026-02-11 12:11:30.322 | INFO     | __main__:select_project:12 - Available projects:


1. PROJECT_ID


Select project number:  1


2026-02-11 12:11:32.161 | INFO     | __main__:select_project:20 - Selected project: PROJECT_ID


<ovhmanager.project.OVHProject>

In [ ]:
#| export
@patch
def list_regions(self:OVHGPUManager)->list:
    """
    Returns the list region for the selected project
    """
    return self.client.get(f'{self.project.base_url}/region')
    

In [ ]:
#| eval: false
client.list_regions()

['BHS',
 'BHS5',
 'CA-EAST-TOR',
 'DE',
 'DE1',
 'EU-SOUTH-MIL',
 'EU-WEST-PAR',
 'GRA',
 'GRA9',
 'RBX',
 'RBX-A',
 'RBX-ARCHIVE',
 'SBG',
 'SBG5',
 'UK',
 'UK1',
 'WAW',
 'WAW1']

In [ ]:
#| export
@patch
def select_region(self:OVHGPUManager, region:str=None)->Any:
    """
    Lists available region for the OVH Project and prompts user to select a region if no region is passed as an argument
    This method sets both the OVHGPUManager and the OVHProject
    
    Parameters
    ----------
    region : str, optional
        OVH region
    """
    if not region:
        logger.info("Available region:")
        regions = self.list_regions()
        for i, p in enumerate(regions):
            print(f"{i+1}. {p}")
        idx = int(input("Select region number: ")) - 1
        region = regions[idx]
    self.region = (region)
    if self.project:
        self.project.set_region(self.region)
    logger.info(f"Selected region: {self.region}")
    return self.region

let's now select a region by passing it

In [ ]:
#| eval: false
client.select_region('GRA9')

2026-02-11 15:20:59.975 | INFO     | __main__:select_region:23 - Selected region: GRA9


'GRA9'

In [ ]:
#| eval: false
client.region, client.project.region

('GRA9', 'GRA9')

Or by interactively selecting it

In [ ]:
#| eval: false
client.select_region()

2026-02-11 12:59:13.263 | INFO     | __main__:select_region:14 - Available region:


1. BHS
2. BHS5
3. CA-EAST-TOR
4. DE
5. DE1
6. EU-SOUTH-MIL
7. EU-WEST-PAR
8. GRA
9. GRA9
10. RBX
11. RBX-A
12. RBX-ARCHIVE
13. SBG
14. SBG5
15. UK
16. UK1
17. WAW
18. WAW1


Select region number:  9


2026-02-11 12:59:17.141 | INFO     | __main__:select_region:23 - Selected region: GRA9


'GRA9'

In [ ]:
#| export
@patch
def select_consumption_mode(self:OVHGPUManager, consumption_mode:str=None)->str:
    """
    Lists available OVH ressources consumption modes and prompts user to select a consumption mode if no mode is passed as an argument

    Parameters
    ----------
    consumption_mode : str, optional
        Ressource consumption mode
    """
    if not consumption_mode:
        logger.info("Available consumption modes:")
        print("\nConsumption modes:")
        print("1. consumption (hourly)")
        print("2. monthly.postpaid")
        mode_choice = int(input("Select mode: "))
        consumption_mode = 'consumption' if mode_choice == 1 else 'monthly.postpaid'
    self.consumption_mode = consumption_mode
    logger.info(f"Selected consumption mode: {self.consumption_mode}")
    return self.consumption_mode

let's now select a consumption mode by passing it

In [ ]:
#| eval: false
client.select_consumption_mode('monthly.postpaid')

2026-02-11 12:11:47.587 | INFO     | __main__:select_consumption_mode:20 - Selected consumption mode: monthly.postpaid


'monthly.postpaid'

In [ ]:
#| eval: false
client.select_consumption_mode()

2026-02-11 12:53:36.534 | INFO     | __main__:select_consumption_mode:13 - Available consumption modes:



Consumption modes:
1. consumption (hourly)
2. monthly.postpaid


Select mode:  1


2026-02-11 12:53:38.138 | INFO     | __main__:select_consumption_mode:20 - Selected consumption mode: consumption


'consumption'

In [ ]:
#| export
@patch
def retrieve_catalog_dict(self:OVHGPUManager, subsidiary='FR') ->dict:
    """
    Retrieves an OVH catalog for a specific OVH subsidiary

    Parameters
    ----------
    subsidiary : str, required, defaults to FR
        OVH catalpg subsidiary/country code
    """
    self.catalog_dict = self.client.get(f'{self._base_catalog_url}', ovhSubsidiary=subsidiary, region=self.region)
    logger.info(f"Retrieved catalog dictionary for {subsidiary} subsidiary")

    return self.catalog_dict
    

In [ ]:
#| eval: false
catalog_dict = client.retrieve_catalog_dict()
assert catalog_dict 

2026-02-11 12:53:42.316 | INFO     | __main__:retrieve_catalog_dict:13 - Retrieved catalog dictionary for FR subsidiary


In [ ]:
#| export
@patch
def select_gpu_instance_type(self:OVHGPUManager, subsidiary='FR', gpu_type=None, consumption_mode=None)->Any:
    """
    Lists available OVH GPU instance types and prompts user to select a GPU type if no gpu type is passed as an argument

    Parameters
    ----------
    subsidiary : str, required, defaults to FR
        OVH catalog subsidiary/country code
    gpu_type : str, optional
        GPU instance type
    consumption_mode : str, optional
        Ressource consumption mode
    """
    # 0. make sure region and projects are set
    if not self.project:
        self.select_project()
    if not self.region:
        self.select_region()
    
    # 1. retrieve the catalog for subsidiary and region combination
    self.retrieve_catalog_dict()
    self.catalog = GPUCatalog(self.catalog_dict, self.region, subsidiary)
    logger.info(f"Retrieved OVH catalog")

    # 2. select consumption mode
    self.select_consumption_mode(consumption_mode)

    if not gpu_type:
        # 3. list available gpu types for 
        self.catalog.list_gpu_instances(self.consumption_mode)
    
        # 4. select gpu instance type
        gpu_idx = int(input("\nSelect GPU instance number: ")) - 1
        gpu_type = self.catalog.gpu_instances_data[gpu_idx]['Name']
        
    self.gpu_type = gpu_type
    logger.info(f"Selected GPU: {self.gpu_type}, with {self.consumption_mode} consumption mode")
    return self.gpu_type

Let's now select a specific gpu type and consumption mode

In [ ]:
#| eval: false
client.select_gpu_instance_type(gpu_type='t2-180', consumption_mode='monthly.postpaid')

2026-02-11 12:53:47.712 | INFO     | __main__:retrieve_catalog_dict:13 - Retrieved catalog dictionary for FR subsidiary
2026-02-11 12:53:47.712 | INFO     | __main__:select_gpu_instance_type:17 - Retrieved OVH catalog
2026-02-11 12:53:47.713 | INFO     | __main__:select_consumption_mode:20 - Selected consumption mode: monthly.postpaid
2026-02-11 12:53:47.713 | INFO     | __main__:select_gpu_instance_type:31 - Selected GPU: t2-180, with monthly.postpaid consumption mode


't2-180'

Let's now interactively select a gpu type

In [ ]:
#| eval: false
client.select_gpu_instance_type()

2026-02-11 12:53:49.314 | INFO     | __main__:retrieve_catalog_dict:13 - Retrieved catalog dictionary for FR subsidiary
2026-02-11 12:53:49.315 | INFO     | __main__:select_gpu_instance_type:17 - Retrieved OVH catalog
2026-02-11 12:53:49.315 | INFO     | __main__:select_consumption_mode:13 - Available consumption modes:



Consumption modes:
1. consumption (hourly)
2. monthly.postpaid


Select mode:  1


2026-02-11 12:53:54.989 | INFO     | __main__:select_consumption_mode:20 - Selected consumption mode: consumption
2026-02-11 12:53:54.996 | INFO     | ovhmanager.catalog:list_gpu_instances:101 - Available GPU instances:


+----+------------+------------------+----------------+--------------+-------------+--------+
|    | Name       | Price per hour   | GPU Model      | GPU Memory   |   CPU Cores | RAM    |
+====+============+==================+================+==============+=============+========+
|  1 | a10-180    | 3.04 €           | A10            | 24GB         |         120 | 180GB  |
+----+------------+------------------+----------------+--------------+-------------+--------+
|  2 | a10-45     | 0.76 €           | A10            | 24GB         |          30 | 45GB   |
+----+------------+------------------+----------------+--------------+-------------+--------+
|  3 | a10-90     | 1.52 €           | A10            | 24GB         |          60 | 90GB   |
+----+------------+------------------+----------------+--------------+-------------+--------+
|  4 | h100-1520  | 11.20 €          | H100           | 80GB         |         120 | 1520GB |
+----+------------+------------------+----------------+-----


Select GPU instance number:  10


2026-02-11 12:54:04.384 | INFO     | __main__:select_gpu_instance_type:31 - Selected GPU: rtx5000-28, with consumption consumption mode


'rtx5000-28'

In [ ]:
#| export
@patch
def retrieve_gpu_flavor(self:OVHGPUManager)->str:
    """
    Retrieve the OVH instance flavor associated the selected GPU type
    """
    try:
        flavors = self.client.get(f'{self.project.base_url}/flavor', region=self.region)
    except Exception:
        print(f'Cannot find ressources for {self.region} region. Please select another region')
        return None
    
    flavor = [f for f in flavors if f['name'] == self.gpu_type][0]
    self.gpu_flavor_id = flavor['id']
    logger.info(f"Selected GPU flavor id: {self.gpu_flavor_id}")
    return self.gpu_flavor_id

In [ ]:
#| eval: false
client.retrieve_gpu_flavor()

2026-02-11 15:08:46.376 | INFO     | __main__:retrieve_gpu_flavor:15 - Selected GPU flavor id: c5708c95-f450-43c3-8b56-7d498e7b5d07


'c5708c95-f450-43c3-8b56-7d498e7b5d07'

In [ ]:
#| export
@patch
def select_instance_image(self:OVHGPUManager, image_family:str='Ubuntu', image_id:str=None, image_name:str=None)->tuple:
    """
    Lists available instance images and prompts user to select an image if no image ID an is passed as an argument

    Parameters
    ----------
    image_family : str, required, defaults to Ubuntu
        The OS familly to look for. Based on key word search
    image_id : str, optional
        The image id if already nown
    image_name : str, optional
        The image name if already nown
    """
    images = self.client.get(f'{self.project.base_url}/image', region=self.region)
    ubuntu_images = [img for img in images if image_family in img['name']]
    if (not image_id) and (not image_name):
        logger.info("\nAvailable Ubuntu images:")
        for i, img in enumerate(ubuntu_images):
            print(f"{i+1}. {img['name']}")
        image_idx = int(input("Select image number: ")) - 1
        selected_image = ubuntu_images[image_idx]
        image_id = selected_image['id']
        image_name = selected_image['name']
    self.instance_image_id = image_id
    self.instance_image_name = image_name
    logger.info(f"Selected instance image: {self.instance_image_id}:{self.instance_image_name}")
    return self.instance_image_id, self.instance_image_name

In [ ]:
#| eval: false
client.select_instance_image()

2026-02-11 12:54:12.964 | INFO     | __main__:select_instance_image:19 - 
Available Ubuntu images:


1. Baremetal - Ubuntu 22.04
2. Ubuntu 25.04
3. Ubuntu 24.04
4. Ubuntu 22.04
5. Ubuntu 25.04 - UEFI
6. Ubuntu 24.04 - UEFI
7. Ubuntu 22.04 - UEFI


Select image number:  3


2026-02-11 12:54:17.841 | INFO     | __main__:select_instance_image:28 - Selected instance image: 8d595649-b9b2-4aeb-a544-a3709abf437f:Ubuntu 24.04


('8d595649-b9b2-4aeb-a544-a3709abf437f', 'Ubuntu 24.04')

In [ ]:
#| export
@patch
def select_ssh_key(self:OVHGPUManager, ssh_key_id:str=None, ssh_key_name:str=None)->tuple:
    """
    Lists available ssh keys and prompts user to select a ssh key if no ssh key ID is passed as an argument

    Parameters
    ----------
    ssh_key_id : str, optional
        OVH ssh key ID
    ssh_key_name: str, optional
        OVH ssh key name
    """
    ssh_keys = self.client.get(f'{self.project.base_url}/sshkey', region=self.region)
    if not ssh_key_id:
        logger.info("\nAvailable SSH keys:")
        for i, key in enumerate(ssh_keys):
            print(f"{i+1}. {key['name']}")
        ssh_idx = int(input("Select SSH key number: ")) - 1
        ssh_key_id = ssh_keys[ssh_idx]['id']
        ssh_key_name = ssh_keys[ssh_idx]['name']
    self.ssh_key_id = ssh_key_id
    self.ssh_key_name =ssh_key_name
    logger.info(f"Selected SSH key: {self.ssh_key_id}:{self.ssh_key_name}")
    return self.ssh_key_id, self.ssh_key_name

In [ ]:
#| eval: false
client.select_ssh_key()

2026-02-11 12:54:22.512 | INFO     | __main__:select_ssh_key:16 - 
Available SSH keys:


1. Mac-V
2. Sys76


Select SSH key number:  1


2026-02-11 12:54:24.946 | INFO     | __main__:select_ssh_key:24 - Selected SSH key: 5457466a4c56593d:Mac-V


('5457466a4c56593d', 'Mac-V')

In [ ]:
#| export
@patch
def create_instance(self:OVHGPUManager, subsidiary='FR', gpu_type=None, consumption_mode=None, image_family:str='Ubuntu',
                    image_id:str=None, image_name:str=None, ssh_key_id:str=None, ssh_key_name:str=None, instance_name=None)->Any:

    """
    Creates an OVH GPU instance either using passed arguments otherwise by prompting the user for inputs

    Parameters
    ----------
    subsidiary : str, required, defaults to FR
        OVH catalpg subsidiary/country code
    gpu_type : str, optional
        GPU instance type
    consumption_mode : str, optional
        Ressource consumption mode
    image_family : str, required, defaults to Ubuntu
        The OS familly to look for. Based on key word search
    image_id : str, optional
        The image id if already nown
    image_name : str, optional
        The image name if already nown
    ssh_key_id : str, optional
        OVH ssh key ID
    ssh_key_name: str, optional
        OVH ssh key name
    instance_name: str, optional
        OVH instance name
    """
    
    self.select_gpu_instance_type(subsidiary, gpu_type, consumption_mode)    
    self.retrieve_gpu_flavor()
    self.select_instance_image(image_family, image_id, image_name)
    self.select_ssh_key(ssh_key_id, ssh_key_name)

    if not instance_name:
        instance_name = input("\nEnter instance name: ")
    self.instance_name = instance_name
    logger.info(f"\nCreating instance '{self.instance_name}'...")
    
    # 5. create instance
    self.last_instance_dict = self.client.post(f'{self.project.base_url}/instance', 
        flavorId=self.gpu_flavor_id,
        imageId=self.instance_image_id,
        name=self.instance_name,
        region=self.region,
        sshKeyId=self.ssh_key_id
    )
    self.last_gpu_instance = GPUInstance(
        client=self.client,
        project_base_url=self.project.base_url,
        region=self.region,
        name=self.last_instance_dict.get('name', 'No Name'),
        status=self.last_instance_dict.get('status', 'No Status'),
        id=self.last_instance_dict.get('id', 'XXXX'),
        gpu_type=self.gpu_type,
        flavor_id=self.gpu_flavor_id,
        image_id=self.instance_image_id,
        consumption_mode=self.consumption_mode,
        created=self.last_instance_dict.get('created', 'No Date')
    )
    return self.last_gpu_instance, self.last_instance_dict

In [ ]:
#| eval: false
client.create_instance()

2026-02-11 15:08:53.885 | INFO     | __main__:retrieve_catalog_dict:13 - Retrieved catalog dictionary for FR subsidiary
2026-02-11 15:08:53.886 | INFO     | __main__:select_gpu_instance_type:19 - Retrieved OVH catalog
2026-02-11 15:08:53.887 | INFO     | __main__:select_consumption_mode:13 - Available consumption modes:



Consumption modes:
1. consumption (hourly)
2. monthly.postpaid


Select mode:  1


2026-02-11 15:08:55.655 | INFO     | __main__:select_consumption_mode:20 - Selected consumption mode: consumption
2026-02-11 15:08:55.661 | INFO     | ovhmanager.catalog:list_gpu_instances:101 - Available GPU instances:


+----+------------+------------------+----------------+--------------+-------------+--------+
|    | Name       | Price per hour   | GPU Model      | GPU Memory   |   CPU Cores | RAM    |
+====+============+==================+================+==============+=============+========+
|  1 | a10-180    | 3.04 €           | A10            | 24GB         |         120 | 180GB  |
+----+------------+------------------+----------------+--------------+-------------+--------+
|  2 | a10-45     | 0.76 €           | A10            | 24GB         |          30 | 45GB   |
+----+------------+------------------+----------------+--------------+-------------+--------+
|  3 | a10-90     | 1.52 €           | A10            | 24GB         |          60 | 90GB   |
+----+------------+------------------+----------------+--------------+-------------+--------+
|  4 | h100-1520  | 11.20 €          | H100           | 80GB         |         120 | 1520GB |
+----+------------+------------------+----------------+-----

...

Select SSH key number:  1


2026-02-11 15:09:05.633 | INFO     | __main__:select_ssh_key:24 - Selected SSH key: 5457466a4c56593d:Mac-V



Enter instance name:  test-auto-6


2026-02-11 15:09:10.821 | INFO     | __main__:create_instance:39 - 
Creating instance 'test-auto-6'...


(<ovhmanager.instance.GPUInstance>,
 {'id': '7ed09818-0b89-4b87-b898-c4a0aa48a1ce',
  'name': 'test-auto-6',
  'ipAddresses': [],
  'status': 'BUILD',
  'created': '2026-02-11T14:09:15Z',
  'region': 'GRA9',
  'flavor': {'id': 'c5708c95-f450-43c3-8b56-7d498e7b5d07',
   'name': 'rtx5000-28',
   'region': 'GRA9',
   'ram': 28,
   'disk': 400,
   'vcpus': 4,
   'type': 'ovh.raid-nvme.t1',
   'osType': 'linux',
   'inboundBandwidth': 2000,
   'outboundBandwidth': 2000,
   'available': True,
   'planCodes': {'monthly': None,
    'hourly': 'rtx5000-28.consumption',
    'license': None},
   'capabilities': [{'name': 'resize', 'enabled': True},
    {'name': 'snapshot', 'enabled': True},
    {'name': 'volume', 'enabled': True},
    {'name': 'failoverip', 'enabled': True}],
   'quota': 7},
  'image': {'id': '8d595649-b9b2-4aeb-a544-a3709abf437f',
   'name': 'Ubuntu 24.04',
   'region': 'GRA9',
   'visibility': 'public',
   'type': 'linux',
   'minDisk': 0,
   'minRam': 0,
   'size': 4,
   'creat

In [ ]:
#| export
@patch
def list_instances(self:OVHGPUManager)->list:
    """
    Lists and displays available OVH GPU instances for specific project
    """
    instances = self.client.get(f'{self.project.base_url}/instance')
    self.gpu_instances = []
    logger.info('\nAvailable GPU instances:')
    for i in instances:
        gpu_type = i.get('planCode').split('.')[0]
        consumption_mode = '.'.join(i.get('planCode').split('.')[1:])
        instance = GPUInstance(
            client=self.client,
            project_base_url=self.project.base_url,
            region=i.get('region'),
            name=i.get('name'),
            status=i.get('status'),
            id=i.get('id'),
            gpu_type=gpu_type,
            flavor_id=i.get('flavorId'),
            image_id=i.get('imageId'),
            consumption_mode=consumption_mode,
            created=i.get('created')
            )
        instance.display_details()
        print(f'------\n')
        self.gpu_instances.append(instance)
        
    return self.gpu_instances

In [ ]:
#| eval: false
client.list_instances()

2026-02-11 15:27:44.939 | INFO     | __main__:list_instances:9 - 
Available GPU instances:


region: GRA9
name: test-auto-6
id: 7ed09818-0b89-4b87-b898-c4a0aa48a1ce
status: ACTIVE
gpu_type: rtx5000-28
flavor_id: c5708c95-f450-43c3-8b56-7d498e7b5d07
image_id: 8d595649-b9b2-4aeb-a544-a3709abf437f
consumption_mode: consumption
planCode: rtx5000-28.consumption
created: 2026-02-11T14:09:15Z
------



[<ovhmanager.instance.GPUInstance>]

In [ ]:
#| eval: false
if len(client.gpu_instances)>0:
    print(client.gpu_instances[0].name, client.gpu_instances[0].id)

test-auto-6 7ed09818-0b89-4b87-b898-c4a0aa48a1ce


In [ ]:
#| export
@patch
def get_instance(self:OVHGPUManager, instance_id:str)->tuple:
    """
    Retrieves an OVH GPU instance information for a specific instance ID

    Parameters
    ----------
    instance_id : str, required
        OVH GPU instance id
    """
    self.last_instance_dict = self.client.get(f'{self.project.base_url}/instance/{instance_id}')
    if self.last_instance_dict:
        gpu_type = None
        consumption_mode = None
        if self.last_instance_dict.get('planCode'):
            gpu_type = self.last_instance_dict.get('planCode').split('.')[0]
            consumption_mode = '.'.join(self.last_instance_dict.get('planCode').split('.')[1:])
        self.last_gpu_instance = GPUInstance(
                client=self.client,
                project_base_url=self.project.base_url,
                region=self.last_instance_dict.get('region'),
                name=self.last_instance_dict.get('name'),
                status=self.last_instance_dict.get('status'),
                id=self.last_instance_dict.get('id'),
                gpu_type=gpu_type,
                flavor_id=self.last_instance_dict.get('flavor')['id'] if self.last_instance_dict.get('flavor') else None,
                image_id=self.last_instance_dict.get('image')['id'] if self.last_instance_dict.get('image') else None,
                consumption_mode=consumption_mode,
                created=self.last_instance_dict.get('created')
                )
        self.last_gpu_instance.display_details()
    return self.last_gpu_instance ,self.last_instance_dict

In [ ]:
#| eval: false
client.get_instance('7ed09818-0b89-4b87-b898-c4a0aa48a1ce')

region: GRA9
name: test-auto-6
id: 7ed09818-0b89-4b87-b898-c4a0aa48a1ce
status: ACTIVE
gpu_type: rtx5000-28
flavor_id: c5708c95-f450-43c3-8b56-7d498e7b5d07
image_id: 8d595649-b9b2-4aeb-a544-a3709abf437f
consumption_mode: consumption
planCode: rtx5000-28.consumption
created: 2026-02-11T14:09:15Z


(<ovhmanager.instance.GPUInstance>,
 {'id': '7ed09818-0b89-4b87-b898-c4a0aa48a1ce',
  'name': 'test-auto-6',
  'ipAddresses': [{'ip': '51.178.114.20',
    'type': 'public',
    'version': 4,
    'networkId': 'bc63b98d13fbba642b2653711cc9d156ca7b404d2df009f7227172d37b5280a6',
    'gatewayIp': '51.178.114.1'},
   {'ip': '2001:41d0:304:400::3b86',
    'type': 'public',
    'version': 6,
    'networkId': 'bc63b98d13fbba642b2653711cc9d156ca7b404d2df009f7227172d37b5280a6',
    'gatewayIp': '2001:41d0:304:400::1'}],
  'status': 'ACTIVE',
  'created': '2026-02-11T14:09:15Z',
  'region': 'GRA9',
  'flavor': {'id': 'c5708c95-f450-43c3-8b56-7d498e7b5d07',
   'name': 'rtx5000-28',
   'region': 'GRA9',
   'ram': 28,
   'disk': 400,
   'vcpus': 4,
   'type': 'ovh.raid-nvme.t1',
   'osType': 'linux',
   'inboundBandwidth': 2000,
   'outboundBandwidth': 2000,
   'available': True,
   'planCodes': {'monthly': None,
    'hourly': 'rtx5000-28.consumption',
    'license': None},
   'capabilities': [{'name

In [ ]:
#| eval: false
#| hide
import nbdev; nbdev.nbdev_export()